# Pre-Processing Forecasting

## Agregat Data $NO_2$ Harian

In [1]:
import pandas as pd

# ====== Konfigurasi file ======
input_path = "dataset/no2_gresik_day.csv"       # ganti dengan path CSV Anda
output_path = "dataset/data.csv"    # file keluaran

# ====== Baca data ======
# parse kolom t sebagai tanggal (YYYY-MM-DD)
df = pd.read_csv(input_path, parse_dates=['t'])

# ====== Hapus kolom yang tidak diperlukan ======
cols_drop = [c for c in ['x', 'y', 'crs'] if c in df.columns]
df = df.drop(columns=cols_drop)

# (opsional) pastikan hanya baris dengan nilai NO2 valid yang dipakai
# df = df.dropna(subset=['NO2'])

# ====== Hitung rata-rata NO2 per hari ======
# Jika 't' berformat tanggal penuh dan Anda HANYA ingin bagian harinya:
df['t'] = df['t'].dt.date  # ubah ke tipe date (tanpa jam)

no2_harian = (
    df.groupby('t', as_index=False)['NO2']
      .mean()
      .rename(columns={'NO2': 'NO2_mean'})
)

# ====== Simpan hasil ======
no2_harian.to_csv(output_path, index=False)

print("Contoh 5 baris teratas:")
print(no2_harian.head())
print(f"\nDisimpan ke: {output_path}")


Contoh 5 baris teratas:
            t  NO2_mean
0  2021-09-20  0.000055
1  2021-09-21       NaN
2  2021-09-22  0.000021
3  2021-09-23  0.000032
4  2021-09-24  0.000033

Disimpan ke: dataset/data.csv


## Penanganan Missing Value dengan Interpolasi Linear

In [2]:
import pandas as pd


# 1) Pastikan kolom t jadi datetime & urut
tmp = no2_harian.copy()
tmp['t'] = pd.to_datetime(tmp['t'])
tmp = tmp.sort_values('t')

# 2) Buat deret tanggal harian kontinu
full_idx = pd.date_range(tmp['t'].min(), tmp['t'].max(), freq='D')
tmp = tmp.set_index('t').reindex(full_idx)
tmp.index.name = 't'

# 3) Interpolasi linear berbasis waktu
tmp['NO2_mean'] = tmp['NO2_mean'].interpolate(method='time')

# 4) Tutup sisa NaN di tepi (awal/akhir) bila ada
tmp['NO2_mean'] = tmp['NO2_mean'].bfill().ffill()

# 5) Kembalikan ke dataframe final (index → kolom t bertipe date opsional)
no2_harian_filled = tmp.reset_index().rename(columns={'index': 't'})
no2_harian_filled['t'] = no2_harian_filled['t'].dt.date  # opsional: jadikan date murni

# Cek
print(no2_harian_filled.head())


            t  NO2_mean
0  2021-09-20  0.000055
1  2021-09-21  0.000038
2  2021-09-22  0.000021
3  2021-09-23  0.000032
4  2021-09-24  0.000033


## Mengubah menjadi data Superviesd

In [4]:
def make_supervised_lag(df_daily):
    df = df_daily.copy()

    # Pastikan tipe datetime & urut
    df['t'] = pd.to_datetime(df['t'])
    df = df.sort_values('t')

    # Normalisasi nama kolom nilai → 'NO_2'
    if 'NO2_mean' in df.columns:
        df = df.rename(columns={'NO2_mean': 'NO_2'})
    elif 'NO2' in df.columns:
        df = df.rename(columns={'NO2': 'NO_2'})
    else:
        raise ValueError("Dataframe harus memiliki kolom 'NO2' atau 'NO2_mean'.")

    # Buat kolom lag (h-1, h-2, h-3) berbasis waktu harian
    df['h-1'] = df['NO_2'].shift(1)
    df['h-2'] = df['NO_2'].shift(2)
    df['h-3'] = df['NO_2'].shift(3)

    # Ambil hanya kolom yang diminta & buang baris awal yang belum punya lag lengkap
    supervised = df[['t', 'h-3', 'h-2', 'h-1', 'NO_2']].dropna(subset=['h-3', 'h-2', 'h-1']).reset_index(drop=True)

    # (opsional) jika ingin 't' sebagai date saja, bukan datetime:
    supervised['t'] = supervised['t'].dt.date

    return supervised

# Pakai fungsi:
df = make_supervised_lag(no2_harian_filled)
print(df.head())

            t       h-3       h-2       h-1      NO_2
0  2021-09-23  0.000055  0.000038  0.000021  0.000032
1  2021-09-24  0.000038  0.000021  0.000032  0.000033
2  2021-09-25  0.000021  0.000032  0.000033  0.000049
3  2021-09-26  0.000032  0.000033  0.000049  0.000035
4  2021-09-27  0.000033  0.000049  0.000035  0.000042


## Scalling Min-Max Normalization

In [15]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

cols_to_scale = [c for c in df.columns if c != 't']

df_scaled = df.copy()
df_scaled[cols_to_scale] = scaler.fit_transform(df_scaled[cols_to_scale])

df_scaled.head()

df = df_scaled.copy()

df.to_csv('dataset/data2.csv',index=False)